# Example submission for HW2
Use this file as a template. In your submission, save any computation results and outputs and submit the file, so we can see it when we grade.

In [ ]:
import random
import time
from copy import deepcopy
from typing import List, Tuple, Optional, Dict, Any, Callable
from tdTTT import TicTacToe3D

class TicTacToeAI: # initialize AI players
    def __init__(self, game: TicTacToe3D, player: str, heuristic_type: str = 'line', max_depth: int = 3, forward_pruning: float = 0.3):
 
        self.game = game # TicTacToe3D game instance
        self.player = player # AI player symbol ('X' or 'O')
        self.opponent = 'O' if player == 'X' else 'X'
        self.max_depth = max_depth # max search depth
        self.forward_pruning = forward_pruning # % of moves to consider (0.0-1.0)
        self.heuristic_type = heuristic_type
        self.nodes_evaluated = 0
        
    def get_move(self) -> Tuple[int, int, int]:
        """Get the best move using alpha-beta search with forward pruning."""
        self.nodes_evaluated = 0
        best_score = float('-inf')
        best_move = None
        alpha = float('-inf')
        beta = float('inf')
        
        legal_moves = self.game.get_legal_moves()
        
        # Apply forward pruning - keep only a percentage of the moves
        pruned_moves = self.prune_moves(legal_moves)
        
        for move in pruned_moves:
            # Create a deep copy of the game to simulate the move
            game_copy = deepcopy(self.game)
            x, y, z = move
            game_copy.make_move(x, y, z)
            
            # The next player is the opponent, so we minimize their score
            score = self.minimax(game_copy, self.max_depth - 1, alpha, beta, False)
            
            if score > best_score:
                best_score = score
                best_move = move
            
            alpha = max(alpha, best_score)
        
        return best_move
    
    def prune_moves(self, moves: List[Tuple[int, int, int]]) -> List[Tuple[int, int, int]]:
        """Apply forward pruning to reduce the number of moves to evaluate."""
        if self.forward_pruning >= 1.0:
            return moves
        
        # Sort moves by heuristic value
        if self.heuristic_type == 'line':
            scored_moves = [(move, self.line_extension_score(move)) for move in moves]
            scored_moves.sort(key=lambda x: x[1], reverse=True)
        else:  # random heuristic
            scored_moves = [(move, random.random()) for move in moves]
            scored_moves.sort(key=lambda x: x[1], reverse=True)
        
        # Keep only a percentage of the highest-scoring moves
        num_moves_to_keep = max(1, int(len(moves) * self.forward_pruning))
        pruned_moves = [move for move, _ in scored_moves[:num_moves_to_keep]]
        
        return pruned_moves
    
    def minimax(self, game: TicTacToe3D, depth: int, alpha: float, beta: float, is_maximizing: bool) -> float:
       
        self.nodes_evaluated += 1
        
        # Check if game is over or max depth reached
        winner = game.check_winner()
        if winner == self.player:
            return 1000 + depth  # Win scores higher if found earlier
        elif winner == self.opponent:
            return -1000 - depth  # Loss scores lower if found earlier
        elif winner == "Draw":
            return 0
        elif depth == 0:
            # Use heuristic evaluation at leaf nodes
            if self.heuristic_type == 'line':
                return self.evaluate_board(game)
            else:  # random heuristic
                return random.uniform(-1, 1)
        
        legal_moves = game.get_legal_moves()
        pruned_moves = self.prune_moves(legal_moves)
        
        if is_maximizing:
            max_eval = float('-inf')
            for move in pruned_moves:
                game_copy = deepcopy(game)
                x, y, z = move
                game_copy.make_move(x, y, z)
                eval = self.minimax(game_copy, depth - 1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break  # Beta cutoff
            return max_eval # best move based on hueristic value
        else:
            min_eval = float('inf')
            for move in pruned_moves:
                game_copy = deepcopy(game)
                x, y, z = move
                game_copy.make_move(x, y, z)
                eval = self.minimax(game_copy, depth - 1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break  # Alpha cutoff
            return min_eval # best move based on hueristic value
    
    def line_extension_score(self, move: Tuple[int, int, int]) -> float: # evaluate how well a move extends existing lines of player's pieces.
    
        game_copy = deepcopy(self.game)
        x, y, z = move
        game_copy.board[z][y][x] = self.player
        
        directions = [
            (1, 0, 0), (0, 1, 0), (0, 0, 1),  # Axes
            (1, 1, 0), (1, 0, 1), (0, 1, 1), (1, -1, 0), (1, 0, -1), (0, 1, -1),  # Face diagonals
            (1, 1, 1), (1, 1, -1), (1, -1, 1), (-1, 1, 1), (-1, -1, -1), (-1, -1, 1), (-1, 1, -1), (1, -1, -1)  # Space diagonals
        ]
        
        score = 0
        size = game_copy.size
        
        for dx, dy, dz in directions:
            # Check both directions
            line_score = 0
            player_count = 0
            opponent_count = 0
            
            for step in range(-(size-1), size):
                nx, ny, nz = x + step * dx, y + step * dy, z + step * dz
                if 0 <= nx < size and 0 <= ny < size and 0 <= nz < size:
                    cell = game_copy.board[nz][ny][nx]
                    if cell == self.player:
                        player_count += 1
                        # Higher score for consecutive pieces
                        if step != 0:  # Don't count the move itself twice
                            line_score += player_count
                    elif cell == self.opponent:
                        opponent_count += 1
                        # Blocking opponent's lines is also valuable
                        if opponent_count >= 2:
                            line_score += 5
            
            score += line_score
            
            # Bonus for almost complete lines
            if player_count == size - 1 and opponent_count == 0:
                score += 50
        
        return score
    
    def evaluate_board(self, game: TicTacToe3D) -> float: # evaluate current board state using the line extension heuristic.
        
        score = 0
        size = game.size
        
        # Check all possible lines
        directions = [
            (1, 0, 0), (0, 1, 0), (0, 0, 1),  # Axes
            (1, 1, 0), (1, 0, 1), (0, 1, 1), (1, -1, 0), (1, 0, -1), (0, 1, -1),  # Face diagonals
            (1, 1, 1), (1, 1, -1), (1, -1, 1), (-1, 1, 1), (-1, -1, -1), (-1, -1, 1), (-1, 1, -1), (1, -1, -1)  # Space diagonals
        ]
        
        for z in range(size):
            for y in range(size):
                for x in range(size):
                    for dx, dy, dz in directions:
                        player_count = 0
                        opponent_count = 0
                        empty_count = 0
                        valid_line = True
                        
                        for step in range(size):
                            nx, ny, nz = x + step * dx, y + step * dy, z + step * dz
                            if 0 <= nx < size and 0 <= ny < size and 0 <= nz < size:
                                cell = game.board[nz][ny][nx]
                                if cell == self.player:
                                    player_count += 1
                                elif cell == self.opponent:
                                    opponent_count += 1
                                else:
                                    empty_count += 1
                            else:
                                valid_line = False
                                break
                        
                        if valid_line:
                            # Score this line
                            if opponent_count == 0:
                                # Line with only our pieces and empty spaces
                                if player_count > 0:
                                    score += 10 ** player_count
                            elif player_count == 0:
                                # Line with only opponent pieces and empty spaces
                                if opponent_count > 0:
                                    score -= 10 ** opponent_count
        
        return score

# run 1 game at a time
def play_game(size: int = 5, max_depth: int = 3, forward_pruning: float = 0.5) -> str:
    
    game = TicTacToe3D(size=size)
    line_ai = TicTacToeAI(game, 'X', heuristic_type='line', max_depth=max_depth, forward_pruning=forward_pruning)
    random_ai = TicTacToeAI(game, 'O', heuristic_type='random', max_depth=max_depth, forward_pruning=forward_pruning)
    
    while game.winner is None:
        current_player = game.check_current_turn()
        if current_player == 'X':
            move = line_ai.get_move()
        else:
            move = random_ai.get_move()
        
        if move:
            x, y, z = move
            game.make_move(x, y, z)
    return game.winner

# simulagte 100 games and print results
def tournament(num_games: int = 100, size: int = 5, max_depth: int = 3, forward_pruning: float = 0.5) -> None:
    
    results = {'X': 0, 'O': 0, 'Draw': 0}
    start_time = time.time()
    
    print(f"Starting tournament: {num_games} games, {size}x{size}x{size} board")
    print(f"Max depth: {max_depth}, Forward pruning: {forward_pruning}")
    print("Line-extension heuristic (X) vs. Random heuristic (O)")
    
    for i in range(num_games):
        if i % 2 == 0: # X (line heuristic) going first
            game = TicTacToe3D(size=size)
            line_ai = TicTacToeAI(game, 'X', heuristic_type='line', max_depth=max_depth, forward_pruning=forward_pruning)
            random_ai = TicTacToeAI(game, 'O', heuristic_type='random', max_depth=max_depth, forward_pruning=forward_pruning)
        else: # O (random heuristic) going first
            game = TicTacToe3D(size=size)
            random_ai = TicTacToeAI(game, 'X', heuristic_type='random', max_depth=max_depth, forward_pruning=forward_pruning)
            line_ai = TicTacToeAI(game, 'O', heuristic_type='line', max_depth=max_depth, forward_pruning=forward_pruning)
        
        while game.winner is None:
            current_player = game.check_current_turn()
            ai = line_ai if current_player == line_ai.player else random_ai
            move = ai.get_move()
            
            if move:
                x, y, z = move
                game.make_move(x, y, z)
        
        # Map the result to the actual heuristic
        if game.winner == 'Draw':
            results['Draw'] += 1
        elif (i % 2 == 0 and game.winner == 'X') or (i % 2 == 1 and game.winner == 'O'):
            results['X'] += 1  # Line heuristic won
        else:
            results['O'] += 1  # Random heuristic won
        
        # Print progress
        if (i + 1) % 10 == 0:
            print(f"Completed {i + 1} games. Current score - Line: {results['X']}, Random: {results['O']}, Draws: {results['Draw']}")
    
    elapsed_time = time.time() - start_time
    
    print("\nTournament Results:")
    print(f"Games played: {num_games}")
    print(f"Line-extension heuristic wins: {results['X']} ({results['X']/num_games*100:.1f}%)")
    print(f"Random heuristic wins: {results['O']} ({results['O']/num_games*100:.1f}%)")
    print(f"Draws: {results['Draw']} ({results['Draw']/num_games*100:.1f}%)")
    print(f"Total time: {elapsed_time:.2f} seconds ({elapsed_time/num_games:.2f} seconds per game)")

if __name__ == "__main__":
    tournament(num_games=100, size=5, max_depth=2, forward_pruning=0.3)

: 

### Player 1: Alpha-beta forward pruning with heuristic 1
Explain here what your heuristic 1 is:

In [ ]:
#implement heuristic 1 and any other functions needed for player 1 here

### Player 2: Alpha-beta forward pruning with heuristic 2
Explain here what your heuristic 2 is:

In [ ]:
#implement heuristic 2 and any other functions needed for player 2 here

### Competition

Run your competition here. Have the two players face each other 100 times, switching who starts each time. Print out the statistics in an easy-to-read format.

In [ ]:
# run and report on competition here
for gameIndex in range(100):
	game = TicTacToe3D(5)
	#run game here...
	print("hi")
	#store results from completed game

#output summary statistics here